In [5]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import locale
locale.setlocale(locale.LC_ALL, 'ID')
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

## GET INDEKS LINK

In [6]:
response = requests.get("https://index.sindonews.com/index/0?t=2018-07-30")
    # Extract HTML texts contained in Response object: html
html = response.text
    # Create a BeautifulSoup object from the HTML: soup
soup = BeautifulSoup(html, "html5lib")
links = []

In [7]:
def getIndeksLink(links, page, cat_link, offset=0, date=datetime.strftime(datetime.today(), '%Y-%m-%d')):
    """
    Untuk mengambil seluruh url 
    link pada indeks category tertentu
    date format : YYYY/mm/dd
    """
    print("page ", page)
    url = "https://index.sindonews.com/index/"+ str(cat_link)+ "/" + str(offset)+ "?t="+ date
    print(url)
    
    # Make the request and create the response object: response
    response = requests.get(url)
    # Extract HTML texts contained in Response object: html
    html = response.text
    # Create a BeautifulSoup object from the HTML: soup
    soup = BeautifulSoup(html, "html5lib")
    
    contentDiv = soup.find('div', class_="indeks-news")
    if contentDiv:
        for post in contentDiv.findAll('div', class_="indeks-title"):
            link = [post.find('a', href=True)['href'], ""]
            links.append(link)
        
    el_page = soup.find('div', class_="pagination")
    if el_page:
        max_page = int(soup.find('div', class_="pagination").findAll('a')[-2]['data-ci-pagination-page'])
            
        if page < max_page:
            links = getIndeksLink(links, page+1, cat_link, offset+10, date)
        
    return links

## GET DETAIL BERITA

In [106]:
response = requests.get("https://autotekno.sindonews.com/read/1324687/123/dominasi-ac-inverter-rumahan-lg-mulai-bidik-pasar-perkantoran-1532443312")
    # Extract HTML texts contained in Response object: html
html = response.text
    # Create a BeautifulSoup object from the HTML: soup
soup = BeautifulSoup(html, "html5lib")
articles = {}

In [111]:
def getDetailBerita(links):
    all_articles = []
    for link in links:
        articles = {}
        #link
        url = link[0]
        response = requests.get(url)
        html = response.text
        # Create a BeautifulSoup object from the HTML: soup
        soup = BeautifulSoup(html, "html5lib")

        #extract subcategory from breadcrumb
        bc = soup.find('ul', class_="breadcrumb")
        if not bc:
            continue
        cat = bc.findAll('a')[-2].text
        sub = bc.findAll('a')[-1].text
            
        #articleid
        #articleid = int(soup.find('article', class_="hentry main read-page--core-article")["data-article-id"])
        #articles['id'] = articleid
        
        #category
        articles['category'] = cat
        articles['subcategory'] = sub
        
        #article_url
        articles['url'] = url        

        #article
        article = soup.find("div", id="content")

        #extract date
        pubdate = soup.find('time').text
        pubdate = pubdate.strip(' \t\n\r')
        pubdate = pubdate.replace(' WIB','')
        articles['pubdate'] = datetime.strftime(datetime.strptime(pubdate, "%A, %d %B %Y - %H:%M"), "%Y-%m-%d %H:%M:%S") 

        #extract author
        reporter = soup.find('div', class_="reporter")
        author = reporter.find('p', class_="author").find('a').text.strip(' ')
        articles['author'] = author
        
        #extract title
        title = soup.find('div', class_="article").find('h1').text
        if ("foto" in title.lower()) or  "video" in title.lower(): 
            continue
        articles['title'] = title   

        #source
        articles['source'] = 'Sindonews'

        #extract comments count
        #comments = soup.find('li', class_="read-page--social-share__list-item js-social-share-comment").find('a')
        #comments = int(comments.find('span', class_="read-page--social-share__comment-total").text)
        #articles['comments'] = comments

        #extract tags
        #tags = soup.findAll('span', class_="tags--snippet__name")
        #tags = ','.join([x.text for x in tags])
        #articles['tags'] = tags

        #extract images
        image = soup.find('div', class_="article").find('img')['src']
        articles['image'] = image
        
        #hapus link sisip image
        for link in article.findAll('img'):
            link.decompose()
            
        #extract content
        detail = BeautifulSoup(article.decode_contents().replace('<br/>', ' '), "html5lib")
        content = re.sub(r'\n|\t|\b|\r','',detail.text)
            
        #articles['content'] = re.sub('google*','', content).strip(' ')
        articles['content'] = content
        #print('memasukkan berita id ', articles['id'])
        all_articles.append(articles)
    return all_articles

In [97]:
link = "https://index.sindonews.com/index/0/0?t=2018-07-30"
links = []

In [116]:
links = getIndeksLink([], 1, 0, 0, '2018-07-30')

page  1
https://index.sindonews.com/index/0/0?t=2018-07-30
page  2
https://index.sindonews.com/index/0/10?t=2018-07-30
page  3
https://index.sindonews.com/index/0/20?t=2018-07-30
page  4
https://index.sindonews.com/index/0/30?t=2018-07-30
page  5
https://index.sindonews.com/index/0/40?t=2018-07-30
page  6
https://index.sindonews.com/index/0/50?t=2018-07-30
page  7
https://index.sindonews.com/index/0/60?t=2018-07-30
page  8
https://index.sindonews.com/index/0/70?t=2018-07-30
page  9
https://index.sindonews.com/index/0/80?t=2018-07-30
page  10
https://index.sindonews.com/index/0/90?t=2018-07-30
page  11
https://index.sindonews.com/index/0/100?t=2018-07-30
page  12
https://index.sindonews.com/index/0/110?t=2018-07-30
page  13
https://index.sindonews.com/index/0/120?t=2018-07-30
page  14
https://index.sindonews.com/index/0/130?t=2018-07-30
page  15
https://index.sindonews.com/index/0/140?t=2018-07-30
page  16
https://index.sindonews.com/index/0/150?t=2018-07-30
page  17
https://index.sindo

In [117]:
links

[['https://metro.sindonews.com/read/1326327/170/usai-bunuh-neneng-roy-bawa-kabur-janda-anak-1-di-tangerang-1532965558',
  ''],
 ['https://ekbis.sindonews.com/read/1326303/34/kebijakan-grab-tidak-rasional-ribuan-pengemudi-berunjuk-rasa-1532958098',
  ''],
 ['https://nasional.sindonews.com/read/1326329/12/usai-bertemu-sby-salim-segaf-merasa-ada-ikatan-batin-yang-kuat-1532968398',
  ''],
 ['https://metro.sindonews.com/read/1326313/170/tiket-asian-games-mahal-pemkab-minta-warga-tangerang-digratiskan-1532960403',
  ''],
 ['https://metro.sindonews.com/read/1326328/170/pembongkaran-jpo-bundaran-hi-ditarget-rampung-satu-malam-1532966985',
  ''],
 ['https://lifestyle.sindonews.com/read/1326191/158/sungyeol-dan-jung-eun-ji-apink-main-film-horor-1532938874',
  ''],
 ['https://autotekno.sindonews.com/read/1326235/120/giias-2018-akan-perkenalkan-teknologi-mobil-hybrid-1532945406',
  ''],
 ['https://ekbis.sindonews.com/read/1326215/178/bri-life-raih-penghargaan-asuransi-jiwa-terbaik-2018-1532943139'

In [118]:
details = getDetailBerita(links[0:3])
details

[{'author': 'Hasan Kurniawan',
  'category': 'Metro',
  'content': 'TANGERANG - Seorang janda anak satu Neneng Nurmaya (35), warga Kampung Panunggulan, Mekarbaru, Kopo, Serang, tewas diduga dibunuh pacarnya, berinisial SH alias Roy, setelah sebulan hidup bersama.  Korban ditemukan tewas kehabisan darah di rumah kontrakannya, Perumahan Taman Adiyasa, Blok J, Desa Cikasungka, Kecamatan Solear, Kabupaten Tangerang.  Kasatreskrim Polresta Tangerang Kompol Wiwin Setiawan menjelaskan, Roy merupakan play boy yang suka menipu janda dengan pendekatan asmara, hanya untuk mendapatkan kepuasan seksual.  "Baik korban dan pelaku masing-masing membawa satu orang anak tinggal di rumah kontrakan itu selama sebulan," kata Wiwin kepada wartawan, Senin (30/7/2018).  Dijelaskan Wiwin, pelaku juga seorang duda. Di rumah kontrakan itu, keduanya membawa masing-masing anaknya. Anak pelaku laki-laki berusia 11 tahun dan anak korban berjenis kelain perempuan berusia 13 tahun.  "Awalnya korban dan pelaku hidup ru

In [119]:
df = pd.DataFrame(details)

In [120]:
df

,author,category,content,image,pubdate,source,subcategory,title,url
0,Hasan Kurniawan,Metro,TANGERANG - Seorang janda anak satu Neneng Nur...,https://cdn.sindonews.net/dyn/620/content/2018...,2018-07-30 23:48:00,Sindonews,Peristiwa,"Usai Bunuh Neneng, Roy Bawa Kabur Janda Anak 1...",https://metro.sindonews.com/read/1326327/170/u...
1,Bona Ventura,Ekbis,JAKARTA - Kebijakan Grab Indonesia menuai keca...,https://cdn.sindonews.net/dyn/620/content/2018...,2018-07-30 23:33:00,Sindonews,Sektor Riil,"Kebijakan Grab Tidak Rasional, Ribuan Pengemud...",https://ekbis.sindonews.com/read/1326303/34/ke...
2,Raka Dwi Novianto,Nasional,JAKARTA - Ketua Majelis Syuro PKS Salim Segaf ...,https://cdn.sindonews.net/dyn/620/content/2018...,2018-07-30 23:33:00,Sindonews,Politik,"Usai Bertemu SBY, Salim Segaf Merasa Ada Ikata...",https://nasional.sindonews.com/read/1326329/12...
